<h1>Content-Based-Filtering</h1>

<h1>پیش پردازش</h1>
<h1>Preprocessing</h1>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

<h2>بارگذاری داده ها</h2>
<h2>Loading datasets</h2>

In [ ]:
data = pd.read_csv("datasets/movies_metadata.csv", low_memory=False)
data.head()

In [ ]:
print(data.shape)

<h3>columns we use</h3>
<h3>ستون هایی که استفاده می کنیم </h3>
<ui>
    <li>genres : ژانر</li>
    <li>original_language : زبان_اصلی</li>
    <li>belongs_to_collection : متعلق_به_مجموعه</li>
    <li>title : عنوان</li>
    <li>vote_average : رای_میانگین</li>
    <li>vote_count : رای_شمارش</li>
    <li>popularity : محبوبیت</li>
    <li>overview : نمای کلی</li>
</ui>

In [4]:
data = data[['id','genres', 'vote_average', 'vote_count','popularity','title',  'belongs_to_collection', 'overview']]

In [ ]:
tmp_m = data['vote_count'].quantile(0.89)
tmp_m

In [ ]:
tmp_data = data.copy().loc[data['vote_count'] >= tmp_m]
tmp_data.shape

In [7]:
del tmp_data

m = data['vote_count'].quantile(0.9)
data = data.loc[data['vote_count'] >= m]

In [ ]:
data.head()

In [ ]:
print(data.shape)

In [ ]:
data = pd.DataFrame.dropna(data, axis=0)
print(data.shape)

In [ ]:
C = data['vote_average'].mean()
print(C)
print(m)

In [12]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    
    return ( v / (v+m) * R ) + (m / (m + v) * C)

In [ ]:
data['score'] = data.apply(weighted_rating, axis = 1)
data.head(5)

In [ ]:
data.shape

In [ ]:
data[['genres', 'belongs_to_collection']].head(2)

<h4>  حل مشکل نوع داده ای ستون های ژانر و متعلق_به_مجموعه</h4>
<br>
<p>برای حل این مشکل از پکیج زیر استفاده میکنیم</p>
<p><a href="https://docs.python.org/3/library/ast.html">ast</a>.literal_eval</p>

In [16]:
data['genres'] = data['genres'].apply(literal_eval)
data['belongs_to_collection'] = data['belongs_to_collection'].apply(literal_eval)

In [ ]:
data[['genres', 'belongs_to_collection']].head(2)

In [18]:
data['genres'] = data['genres'].apply(lambda x : [d["name"] for d in x]).apply(lambda x : "".join(x))
data['belongs_to_collection'] = data['belongs_to_collection'].apply(lambda x : [d[:1]for d in x]).apply(lambda x : "".join(x))

In [ ]:
data.head(2)

<h1>Content-Based-Filtering</h1>
<h1>فیلترینگ مبتنی بر محتوا</h1>
<h3>می خواهیم بر اساس محتوا توصیه هایی ارائه کنیم. با استفاده از فیلترینگ مبتنی بر محتوا به توصیه محتوای مشابه به کاربران اشاره کنیم
 </h3>
 <h3>ژانر میتواند یک گزینه مناسب برای توصیه باشد</h3>

In [ ]:
movie_data = pd.read_csv('datasets/movies_metadata.csv', low_memory=False)
movie_data =  movie_data.loc[movie_data['original_language'] == 'en', :]
movie_data = movie_data[['id', 'title', 'original_language', 'genres']]

print(movie_data.shape)
movie_data.head()

In [ ]:
movie_keyword = pd.read_csv('datasets/keywords.csv', low_memory=False)
print(movie_keyword.shape)
movie_keyword.head()

In [ ]:
movie_data.id = movie_data.id.astype(int)
movie_keyword.id = movie_keyword.id.astype(int)
movie_data = pd.merge(movie_data, movie_keyword, on='id')
print(movie_data.shape)
movie_data.head()

<h1>پیش پردازش</h1>
<h1>Preprocessing</h1>

In [23]:
movie_data['genres'] = movie_data['genres'].apply(literal_eval)
movie_data['genres'] = movie_data['genres'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))

In [24]:
movie_data['keywords'] = movie_data['keywords'].apply(literal_eval)
movie_data['keywords'] = movie_data['keywords'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))

In [ ]:
movie_data.head()

<h2>داده های پیش پردازش شده با استفاده از روش TF-IDF به بردار تبدیل می شوند.
من ژانر و کلمه کلیدی را با هم ترکیب کردم و آن را به یک وکتور tfidf تبدیل کردم.</h2>

In [26]:
tfidf_vector = TfidfVectorizer()
#tfidf_vector = TfidfVectorizer(ngram_range=(1,2))
tfidf_matrix = tfidf_vector.fit_transform(movie_data['genres'] + " " + movie_data['keywords']).toarray()
#tfidf_matrix = tfidf_vector.fit_transform(movie_data['genres']).toarray()
tfidf_matrix_feature = tfidf_vector.get_feature_names_out()

In [ ]:
tfidf_matrix.shape

In [ ]:
tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=tfidf_matrix_feature, index = movie_data.title)
print(tfidf_matrix.shape)
tfidf_matrix.head()

In [ ]:
%%time
cosine_sim = cosine_similarity(tfidf_matrix)

In [ ]:
cosine_sim.shape

In [ ]:
cosine_sim_df = pd.DataFrame(cosine_sim, index = movie_data.title, columns = movie_data.title)
print(cosine_sim_df.shape)
cosine_sim_df.head()

In [32]:
def genre_recommendations(target_title, matrix, items, k=10):
    recom_idx = matrix.loc[:, target_title].values.reshape(1, -1).argsort()[:, ::-1].flatten()[1:k+1]
    recom_title = items.iloc[recom_idx, :].title.values
    recom_genre = items.iloc[recom_idx, :].genres.values
    target_title_list = np.full(len(range(k)), target_title)
    target_genre_list = np.full(len(range(k)), items[items.title == target_title].genres.values)
    d = {
        'target_title':target_title_list,
        'target_genre':target_genre_list,
        'recom_title' : recom_title,
        'recom_genre' : recom_genre
    }
    return pd.DataFrame(d)

In [ ]:
genre_recommendations('The Dark Knight Rises', cosine_sim_df, movie_data)

<h1>Level Of code : Basic</h1>